In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import math
import random
import glob, os
from pathlib import Path
from PIL import Image


from dataloader import DataLoader
#from torch.utils.data import DataLoader


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.nn.functional as F
import torch.optim as optim
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard  import SummaryWriter

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits import mplot3d
from numpy import linalg as LA
import sklearn.metrics as metrics

import trimesh
import meshplot as mp

from DGCNN_simple import DGCNN_simple, DGCNN_reg
from data_depth import DepthDataset
import random
import time

from sklearn.metrics import f1_score,precision_score,recall_score,balanced_accuracy_score

In [2]:
def intrinsic_from_fov(height, width, fov=90):
    """
    Basic Pinhole Camera Model
    intrinsic params from fov and sensor width and height in pixels
    Returns:
        K:      [4, 4]
    """
    px, py = (width / 2, height / 2)
    hfov = fov / 360. * 2. * np.pi
    fx = width / (2. * np.tan(hfov / 2.))

    vfov = 2. * np.arctan(np.tan(hfov / 2) * height / width)
    fy = height / (2. * np.tan(vfov / 2.))

    return np.array([[fx, 0, px, 0.],
                     [0, fy, py, 0.],
                     [0, 0, 1., 0.],
                     [0., 0., 0., 1.]])

def plot_3d(pixels, depth, img_shape, point_size=0.1, zNear=0.85, zFar=4.0, fov_deg=49.1):

    indices = torch.tensor(pixels.T, dtype=torch.int64)
    z_3d = torch.tensor(depth)
    
    depth = 2.0 * zNear * zFar / (zFar + zNear - z_3d * (zFar - zNear))
    
    height, width = img_shape
    K = intrinsic_from_fov(height, width, fov_deg)

    cam_coords = []
    u0 = K[0, 2]
    v0 = K[1, 2]
    fx = K[0, 0]
    fy = K[1, 1]

    # Loop through each pixel
    for i, (v, u) in enumerate(indices):
            # Apply equation in fig 3
            x = -(v - v0) * depth[i] / fx
            y =  (u - u0) * depth[i] / fy
            z = -depth[i]
            cam_coords.append([y.item(), x.item(), z.item()])
    cam_coords_tensor = torch.FloatTensor(cam_coords)

    mp.plot(cam_coords_tensor.numpy(), c=depth.numpy(), shading={"point_size": point_size,"colormap": "plasma_r"})

In [3]:
#def evaluate(model, dataloader, criterion1, criterion2, device):
def evaluate(model, dataloader, criterion1, device):
    model.eval()
    num_val_batches = len(dataloader)
    val_loss1 = 0
    #val_loss2 = 0
    #for pixels, depth, knn in dataloader:
    for pixels, depth in dataloader:
        #pixels, depth, knn = pixels.to(device), depth.to(device), knn.to(device)
        pixels, depth = pixels.to(device), depth.to(device)
        
        with torch.no_grad():
            depth_pred = model(pixels)
            #depth_filt = torch.matmul(knn, depth_pred.unsqueeze(2)).squeeze(2)
            loss1      = criterion1(depth_pred, depth)
            #loss2      = criterion2(depth_pred, depth_filt)
    
        val_loss1 += loss1.item()
        #val_loss2 += loss2.item()
    
    model.train()
    return val_loss1/num_val_batches #, val_loss2/num_val_batches

In [4]:
# Meta
device = torch.device("cuda")

num_points = 2000
batch_size = 12

use_validation = True

#val_percent = 0.1
val_batch_size = batch_size


In [5]:
device

device(type='cuda')

In [6]:
dataset_train = DepthDataset(num_points, normalize=True, partition='train', repeat=1)
dataset_valid = DepthDataset(num_points, normalize=True, partition='valid', repeat=1)

#pixels, depth, knn = dataset_train[5]
pixels, depth = dataset_train[5]
plot_3d(dataset_train.denormalize(pixels), depth, dataset_train.im_shape)

C:\Users\GASTN~1\AppData\Local\Temp/ipykernel_5964/4183062547.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[fx, 0, px, 0.],


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.036698…

In [7]:
# Dataloaders
#n_val = int(len(dataset)*val_percent) if use_validation else 0
#n_test = int(len(dataset)*test_percent) if use_validation else 0
#n_train = len(dataset) - n_val - n_test

#train_set, val_set, test_set = random_split(dataset, [n_train, n_val, n_test], generator=torch.Generator().manual_seed(0))

train_loader = DataLoader(dataset_train, num_workers=8, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
if use_validation:
    val_loader = DataLoader(dataset_valid, num_workers=8, batch_size=val_batch_size, shuffle=False, drop_last=True, pin_memory=True)
    test_loader = DataLoader(dataset_valid, num_workers=1, batch_size=1, shuffle=False, drop_last=False, pin_memory=True)

In [8]:
# Create model - defaults: k=20, emb_dims=1024, dropout=0.5
#model = DGCNN_simple(k=20, emb_dims=1024, dropout=0.5, output_channels=num_points).to(device)
model = DGCNN_reg(k=20, emb_dims=1024, dropout=0.5, output_channels=1).to(device)

In [9]:
# Load pretrained
#model.load_state_dict(torch.load("models/model_state_dict1_1200"), strict=False)
#model.load_state_dict(torch.load("models/model_state_dict_2000puntos_sigmoid_2000epochs"))
model.load_state_dict(torch.load("checkpoints/checkpoint_epoch79.pth"))

<All keys matched successfully>

In [10]:
model = nn.DataParallel(model)
print("Let's use", torch.cuda.device_count(), "GPUs!")
print('N° params:', sum(p.numel() for p in model.parameters() if p.requires_grad))

N° params: 979584


In [11]:
# Optimizer
lr        = 0.001
epochs    = 1000
opt       = optim.Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, 1000, eta_min=lr)

criterion1 = nn.MSELoss()
#criterion2 = nn.MSELoss()


In [12]:
len(train_loader)

1800

In [13]:
writer = SummaryWriter('runs/experiment_24k_2')

steps = len(train_loader)
last_printed = -1
for epoch in range(epochs):
    
    print('Epoch', epoch, 'of', epochs)
    
    train_loss1 = 0.0
    #train_loss2 = 0.0
    count = 0
    model.train()
    
    dt_model = []
    dt_loader = []
    t2 = time.perf_counter()
    
    # Training loop
    #for pixels, depth, knn in train_loader:
    for pixels, depth in train_loader:
        
        t1 = time.perf_counter()
        dt_loader.append(t1-t2)
        #print('    Loader:', t1-t2)
        
        if (count+1) % (steps//4) == 0:
            print('  Epoch percent:', 100*(count+1)/steps, '%',
                  '- step', count+1, '/', len(train_loader), 
                  '- avg time loader:', np.round(np.mean(dt_loader), 3),
                  '- avg time model:', np.round(np.mean(dt_model), 3))
            dt_model = []
            dt_loader = []
            
        # get pixels and depth data
        #pixels, depth, knn = pixels.to(device), depth.to(device), knn.to(device)
        pixels, depth = pixels.to(device), depth.to(device)
        batch_size  = pixels.size()[0]
                
        # model pass
        opt.zero_grad()
        depth_pred = model(pixels)
        #depth_filt = torch.matmul(knn, depth_pred.unsqueeze(2)).squeeze(2)
        loss1      = criterion1(depth_pred, depth)
        #loss2      = 0.0*criterion2(depth_pred, depth_filt)
        loss       = loss1 #+ loss2
        loss.backward()
        opt.step()
        
        # metrics
        count += 1
        train_loss1 += loss1.item()
        #train_loss2 += loss2.item()
        
        #t2_prev = t2
        t2 = time.perf_counter()
        dt_model.append(t2-t1)
        #print('    Model:', t2-t1)
        #print('Total:', t2-t2_prev)
        
        
    scheduler.step()
    writer.add_scalar('Learning rate', opt.param_groups[0]['lr'], epoch)
    
    # Log the losses
    if use_validation:
        #val_loss1, val_loss2 = evaluate(model, val_loader, criterion1, criterion2, device)
        val_loss1 = evaluate(model, val_loader, criterion1, device)
        writer.add_scalars('Loss/Info per epoch', {'Train1': train_loss1/count, #'Train2': train_loss2/count,
                                                   'Validation1': val_loss1}, epoch) #, 'Validation2': val_loss2}, epoch)
    else:
        writer.add_scalars('Loss/Info per epoch', {'Train1': train_loss1/count, # 'Train2': train_loss2/count,
                                                   'Validation1': 0}, epoch) #, 'Validation2': 0}, epoch)
    if True:
        checkpoint_path = Path('./checkpoints/')
        checkpoint_path.mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), str(checkpoint_path / 'checkpoint_epoch{}.pth'.format(epoch + 1)))
        
    writer.flush()

print('Done!')

Epoch 0 of 1000


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.034 - avg time model: 0.348
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.035 - avg time model: 0.35
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.035 - avg time model: 0.346
Epoch 1 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.03 - avg time model: 0.35
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.344
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.341
Epoch 2 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.341
Epoch 3 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 50.0 % - step 900 / 1800 

  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.341
Epoch 29 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.341
Epoch 30 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.34
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.341
Epoch 31 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.029 - avg time model: 0.341
  Epoch percent: 75.0 % - step 1350 /

  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.343
Epoch 57 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.347
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.03 - avg time model: 0.345
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.029 - avg time model: 0.343
Epoch 58 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.031 - avg time model: 0.363
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.03 - avg time model: 0.354
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.03 - avg time model: 0.354
Epoch 59 of 1000
  Epoch percent: 25.0 % - step 450 / 1800 - avg time loader: 0.029 - avg time model: 0.34
  Epoch percent: 50.0 % - step 900 / 1800 - avg time loader: 0.03 - avg time model: 0.354
  Epoch percent: 75.0 % - step 1350 / 1800 - avg time loader: 0.03 - avg time model: 0.347
Epoch 60 of 1000
  Epoch percent: 25.0 % 

KeyboardInterrupt: 

In [ ]:
filename = "models/model_state_dict_24k_1000epochs_k60_std4_lambda0"
if os.path.exists(filename):
    raise Exception('File name already exists')
torch.save(model.state_dict(), filename)

In [ ]:
# Load and test model
#model = DGCNN_reg(k=20, emb_dims=1024, dropout=0.5, output_channels=1).to(device)
#model.load_state_dict(torch.load("models/model_state_dict_overfit_orig_600epochs_loss_skeleton_k60_std4_lambda01"))

In [12]:
model.eval()

loader_iter = iter(test_loader) if use_validation else iter(dataset_train)

In [64]:
type(loader_iter)

dataloader.DataLoader

In [258]:
#pixels, depth, knn = next(loader_iter)
pixels, depth = next(loader_iter)

pixels = pixels.to(device)
depth_pred = model(pixels)

pixels_numpy = pixels.detach().cpu().squeeze().numpy()
pixels_denorm = dataset_valid.denormalize(pixels_numpy)
depth_pred_numpy = depth_pred.detach().cpu().squeeze().numpy()

plot_3d(pixels_denorm, depth_pred_numpy, dataset_valid.im_shape, point_size=0.08)

C:\Users\GASTN~1\AppData\Local\Temp/ipykernel_5964/4183062547.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[fx, 0, px, 0.],


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001047…

In [ ]:
# Test a new image

input_img = 'C:/Users/Gastón/Desktop/Gaston/CONICET/repos/dgcnn_emma/test_imgs/test2.jpg'

# Load image
img = np.array(Image.open(input_img))[:,:,0]

# Get array of pixels locations
r,c = np.where(img < 50)
pixels = np.vstack((r[:,np.newaxis].T, c[:,np.newaxis].T))

# Shuffle pixels and select first N
np.random.shuffle(pixels.T)
pixels = pixels[:,:num_points]

# Normalize to [-1,1]
pixels = 2 * pixels.astype(np.double) / np.array(img.shape)[:,np.newaxis] - 1


pixels = torch.Tensor(pixels).unsqueeze(0).to(device)
depth_pred = model(pixels)

pixels_numpy = pixels.detach().cpu().squeeze().numpy()
pixels_denorm = dataset.denormalize(pixels_numpy)
depth_pred_numpy = depth_pred.detach().cpu().squeeze().numpy()

plot_3d(pixels_denorm, depth_pred_numpy, dataset.im_shape)
